In [1]:
# import pytesseract
# img = '/home/mnemonic/work/digits/real_source_2/21.jpg'
# custom_config = r'--oem 3 --psm 6 outputbase digits'
# print(pytesseract.image_to_string(img, config=custom_config))

# read hid device python
# https://www.ontrak.net/pythonhidapi.htm
# https://pypi.org/project/hid/

# full screen opencv
# https://answers.opencv.org/question/198479/display-a-streamed-video-in-full-screen-opencv3/

In [1]:
import pyzbar.pyzbar as pyzbar
import numpy as np
import mysql.connector
import threading
import cv2
import time
import logging
from datetime import datetime
from contextlib import closing

In [2]:
def create_table(config):
    mydb = mysql.connector.connect(**config)
    with closing(mydb) as conn:
        with conn.cursor() as cursor:
            cursor.execute('CREATE TABLE IF NOT EXISTS qr_table (id INT NOT NULL AUTO_INCREMENT, data VARCHAR(100) NOT NULL, time TIMESTAMP NOT NULL, route VARCHAR(10) NOT NULL, check_flag INT NOT NULL, PRIMARY KEY (id));')
            conn.commit()

def push_to_db(qr_dict, config):
    mydb = mysql.connector.connect(**config)
    with closing(mydb) as conn:
        with conn.cursor() as cursor:
            placeholders = ', '.join(['%s'] * len(qr_dict))
            columns = ', '.join(qr_dict.keys())
            sql = "INSERT INTO %s ( %s ) VALUES ( %s )" % ('qr_table', columns, placeholders)
            cursor.execute(sql, list(qr_dict.values()))
            conn.commit()
            rootLogger.info(f'{qr_dict} data have been writed to db')

In [3]:
def registrate_new_qr(decodedObjects, STOCK_LAST_DATA, camera_label):
    
    min_size_stock_last_data = 3
    max_size_stock_last_data = 7
    default_check_flag = 0    
    current_data_list = [x.data.decode("utf-8") for x in decodedObjects]
    
    # push new data to db  
    if not any(item in current_data_list for item in STOCK_LAST_DATA):
        for obj in decodedObjects:
            # init dict with current data
            now = datetime.now()
            new_data = {'data': obj.data.decode("utf-8"),
                      'time': now.strftime("%Y/%m/%d %H:%M:%S"),
                      'route': camera_label,
                      'check_flag': default_check_flag}
            # insert new qr code with time registration to qr_table qr database
            push_to_db(new_data, DB_CONFIG)
            
    # add new data to STOCK_LAST_DATA
    for obj in decodedObjects:
        if obj.data.decode("utf-8") not in STOCK_LAST_DATA: 
            STOCK_LAST_DATA.append(obj.data.decode("utf-8"))
            debugLogger.debug(f'{obj.data.decode("utf-8")} have been append to STOCK_LAST_DATA')
            
    # clean STOCK_LAST_DATA and pass only min_size_stock_last_data elements
    if len(STOCK_LAST_DATA)>max_size_stock_last_data:
        STOCK_LAST_DATA = STOCK_LAST_DATA[-min_size_stock_last_data:]
        debugLogger.debug(f'delite part of data from stock. Current stock: {STOCK_LAST_DATA}')
        
    return STOCK_LAST_DATA

In [4]:
def add_qr_to_frame(frame, decodedObjects):
    
    for decodedObject in decodedObjects:        
        # Drow square around qr
        points = decodedObject.polygon
        # If the points do not form a quad, find convex hull
        if len(points) > 4 : 
          hull = cv2.convexHull(np.array([point for point in points], dtype=np.float32))
          hull = list(map(tuple, np.squeeze(hull)))
        else : 
          hull = points
        # Number of points in the convex hull
        n = len(hull)     
        # Draw the convext hull
        for j in range(0,n):
          cv2.line(frame, hull[j], hull[ (j+1) % n], (0,0,255), 3)
        
        # Show text
        x = decodedObject.rect.left
        y = decodedObject.rect.top
        barCode = str(decodedObject.data.decode("utf-8"))
        cv2.putText(frame, barCode, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

In [5]:
def process_stream(STREAM_URL, camera_label, DB_CONFIG):
    
    # Create mysql table for qr data if not exist
    create_table(DB_CONFIG)

    # Get the webcam or stream: 
    if STREAM_URL == '':
        cap = cv2.VideoCapture(0)
    else:
        cap = cv2.VideoCapture(STREAM_URL)
        
    # Resize cap axample size 1024.0 x 768.0; 1280.0 x 1024.0
    cap.set(3,640)
    cap.set(4,480)
    time.sleep(2)

    # Init stock - short memory for qr data
    STOCK_LAST_DATA = []

    # Endless cycle
    while(cap.isOpened()):
        # Capture frame-by-frame
        ret, frame = cap.read()
        # Fix time
        last_time = time.time()
        # Our operations on the frame come here
        im = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
        # Processing QR in frame
        decodedObjects = pyzbar.decode(im)              
        if decodedObjects:
            # Update stock and push to db if not in stock 
            STOCK_LAST_DATA = registrate_new_qr(decodedObjects, STOCK_LAST_DATA, camera_label)
            # Add square arround qr and text with data on frame 
            add_qr_to_frame(frame, decodedObjects)
        # Display FPS:
        cv2.putText(frame, "FPS: %f" % (1.0 / (time.time() - last_time)),
                                (10, 20),  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        # Display the resulting frame
        cv2.imshow(camera_label+'_frame',frame)
        # Control keys
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q'):
            break
        elif key & 0xFF == ord('s'): # wait for 's' key to save 
            cv2.imwrite('Capture.png', frame)     

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

In [6]:
# Global variebles
# Example "rtsp://192.168.1.2:8080/out.h264" if empty() - use front notebook camera
RIGHT_STREAM = 'rtsp://wowzaec2demo.streamlock.net/vod/mp4:BigBuckBunny_115k.mov' 
LEFT_STREAM = ''

# Side labels. There are 'names' of cameras. We have 2 cameras in system: 'left' and 'right' now
RIGHT_LABEL = 'right'
LEFT_LABEL = 'left'

# DB config. You should create qr database, but qr_table table will create automatically if it not exist 
DB_CONFIG = {
    'host': "localhost",
    'user': "mysql_user",
    'password': "mysql_password",
    'database': "qr"}


if __name__ == '__main__':
    
    # Logger
    logFormatter = logging.Formatter("%(asctime)s [%(threadName)-12.12s] [function %(funcName)s] [%(levelname)-5.5s] %(message)s")
    global rootLogger
    rootLogger = logging.getLogger('root')
    rootLogger.setLevel(logging.INFO)
    fileHandler_root = logging.FileHandler("log.log")
    fileHandler_root.setFormatter(logFormatter)
    rootLogger.addHandler(fileHandler_root)
    global debugLogger
    debugLogger = logging.getLogger('debug')
    debugLogger.setLevel(logging.DEBUG)
    fileHandler_debug = logging.FileHandler("debug.log")
    fileHandler_debug.setFormatter(logFormatter)
    debugLogger.addHandler(fileHandler_debug)
    
    try:
        # Process streams
        thread_right = threading.Thread(target=process_stream, args=(RIGHT_STREAM, RIGHT_LABEL, DB_CONFIG, )) 
        thread_left = threading.Thread(target=process_stream, args=(LEFT_STREAM, LEFT_LABEL, DB_CONFIG, )) 
        thread_right.start()
        rootLogger.info('thread_right have been started')
        thread_left.start()
        rootLogger.info('thread_left have been started')
        thread_right.join()
        rootLogger.info('thread_right have been stoped')
        thread_left.join()
        rootLogger.info('thread_left have been stoped')
    except Exception as e:
        debugLogger.exception(e)
        #     rootLogger.info("Exception occurred")


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-e0a5b8067553>", line 27, in process_stream
cv2.error: OpenCV(4.2.0) ../modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'



In [7]:
! ls -la

total 148
drwxrwxr-x 5 mnemonic mnemonic  4096 Nov  9 10:58 .
drwxrwxr-x 4 mnemonic mnemonic  4096 Oct  7 11:21 ..
-rw-rw-r-- 1 mnemonic mnemonic 16655 Nov  9 10:59 debug.log
drwxrwxr-x 2 mnemonic mnemonic  4096 Oct 31 15:46 .ipynb_checkpoints
-rw-rw-r-- 1 mnemonic mnemonic 57084 Nov  9 11:00 log.log
-rw-rw-r-- 1 mnemonic mnemonic 30438 Nov  4 13:47 python_libs_3_datarut.ipynb
drwxr-xr-x 2 mnemonic mnemonic  4096 Oct  3 16:46 real_source
drwxrwxr-x 2 mnemonic mnemonic  4096 Oct 27 16:08 real_source_2
-rw-rw-r-- 1 mnemonic mnemonic 11503 Nov  9 10:58 tests_freshpak.ipynb


In [1]:
! tail debug.log

2020-11-09 10:59:22,462 [Thread-25   ] [function registrate_new_qr] [DEBUG] https://www.dropbox.com/s/705b6p4a2ydvayx/EN-Poster.pdf?dl=0 have been append to STOCK_LAST_DATA
2020-11-09 10:59:22,462 [Thread-25   ] [function registrate_new_qr] [DEBUG] https://www.dropbox.com/s/705b6p4a2ydvayx/EN-Poster.pdf?dl=0 have been append to STOCK_LAST_DATA
2020-11-09 10:59:22,462 [Thread-25   ] [function registrate_new_qr] [DEBUG] https://www.dropbox.com/s/705b6p4a2ydvayx/EN-Poster.pdf?dl=0 have been append to STOCK_LAST_DATA
2020-11-09 10:59:22,462 [Thread-25   ] [function registrate_new_qr] [DEBUG] https://www.dropbox.com/s/705b6p4a2ydvayx/EN-Poster.pdf?dl=0 have been append to STOCK_LAST_DATA
2020-11-09 10:59:22,462 [Thread-25   ] [function registrate_new_qr] [DEBUG] https://www.dropbox.com/s/705b6p4a2ydvayx/EN-Poster.pdf?dl=0 have been append to STOCK_LAST_DATA
2020-11-09 10:59:22,462 [Thread-25   ] [function registrate_new_qr] [DEBUG] https://www.dropbox.com/s/705b6p4a2ydvayx/EN-Poster.pdf?dl=